# Mesh Centres

In [1]:
import sys
import plotly.express as px
import pandas as pd
import geopandas as gdp
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import math
import numpy as np
import sys
sys.path.append('../')
from api import MAPBOX_ACCESSTOKEN
from Utilities.open_map import mesh_map, regular_map, sa3_map, sa2_map, sa4_map

In [2]:
# The mesh map takes a while to load
shape_json, shape = sa4_map() 
lga_json, df_map = regular_map() 

In [3]:
print(shape.columns.values.tolist())

['SA4_CODE21', 'SA4_NAME21', 'CHG_FLAG21', 'CHG_LBL21', 'GCC_CODE21', 'GCC_NAME21', 'STE_CODE21', 'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21', 'AREASQKM21', 'LOCI_URI21', 'geometry']


In [4]:

f = open("./SA4_centres.txt", "w", encoding="utf-8")
f.write('LATITUDE\tLONGITUDE\tLOCI_URI21\tAUS_CODE21\tGCC_CODE21\tGCC_NAME21\tAUS_NAME21\tCHG_LBL21\tCHG_FLAG21\tSTE_NAME21\tSTE_CODE21\tAREASQKM21\tSA4_CODE21\tSA4_NAME21\n')

geos = shape['geometry']
LOCI = shape['LOCI_URI21']
AUS_CODE21 = shape['AUS_CODE21']
GCC_CODE = shape['GCC_CODE21']
GCC_NAME = shape['GCC_NAME21']
AUS_NAME = shape['AUS_NAME21']
CHG_LBL = shape['CHG_LBL21']
CHG_FLAG = shape['CHG_FLAG21']
STE_NAME = shape['STE_NAME21']
STE_CODE = shape['STE_CODE21']
AREASQK = shape['AREASQKM21']
SA4_CODE = shape['SA4_CODE21']
SA4_NAME = shape['SA4_NAME21']

no_centroids = np.array([])
for geo, loci, aus_code, gcc_code, gcc_name, aus_name, chg_lbl, chg_flag, ste_name, ste_code, areasqk, sa4_code, sa4_name in zip(geos, LOCI,AUS_CODE21, GCC_CODE, GCC_NAME, AUS_NAME, CHG_LBL, CHG_FLAG, STE_NAME, STE_CODE, AREASQK, SA4_CODE, SA4_NAME):
    try:
        centroid = geo.centroid 
        lat = centroid.y
        lon = centroid.x
        f.write(f"{lat}\t{lon}\t{loci}\t{aus_code}\t{gcc_code}\t{gcc_name}\t{aus_name}\t{chg_lbl}\t{chg_flag}\t{ste_name}\t{ste_code}\t{areasqk}\t{sa4_code}\t{sa4_name}\n")
    except Exception as ex:
        no_centroids = np.append(no_centroids, sa4_name)
f.close()

# Now write in the values with no centroids
f = open("./no_sa4_centres.txt", "w", encoding="utf-8")
f.write('SA4\n')
for code in no_centroids:
    f.write(f"{code}\n")
f.close()

        
    

In [ ]:
sa4 = pd.read_csv("./SA4_centres.txt",
                             sep='\\t',
            encoding='unicode_escape',
            engine='python')

# GHet the mesh centroids 
sa4_centres_latitudes = np.array(sa4['LATITUDE'])
sa4_centres_longitudes = np.array(sa4['LONGITUDE'])
sa4_centre_names = np.array(sa4['SA3_CODE21'])
sa4_centre_codes = np.array(sa4['STE_CODE21'])



# Map the regular map 
trace1 =  px.choropleth_mapbox(geojson=lga_json, locations =df_map.index, mapbox_style='light')

# Create a map of centroids 
trace1.add_trace(go.Scattermapbox(lat=sa4_centres_latitudes,lon=sa4_centres_longitudes, name='Mesh Centroids',  mode='markers', marker=go.scattermapbox.Marker(size=5, color='red'), text=sa4_centre_names
))


trace1.update_layout(
    title={'text': f"NSW",
        'font': {'size': 24}},
    mapbox=dict(
        center=dict(lat=-33.8603, lon=151.0236),
        accesstoken=MAPBOX_ACCESSTOKEN,
        zoom=9.5,
    ),
    autosize=True,
    height=650,
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1,
        x=0,
        y=1,
        orientation='v'
    ))



